**Rust Quantum Library**

Below are dependencies, creates, and global variables.

In [2]:
:dep ndarray = { version = "0.14", features = ["serde"] }
:dep ndarray-linalg = { version = "0.13.1", features = ["intel-mkl-static"]}
:dep plotters = { git = "https://github.com/38/plotters", default_features = false, features = ["evcxr", "all_series"] }
:dep thiserror = "1.0"
:dep serde = { version = "1.0", features = ["derive"] }
:dep serde_json = "1.0"
:dep dimensioned = "0.7"
:dep num-complex = "0.4"
:dep qm = { version = "0.1", git ="https://github.com/buff-beacon-project/qm" }
:dep ndarray-csv = "0.5.1"
extern crate plotters;
extern crate ndarray;

pub use ndarray_linalg::c64;
use std::f64::consts::PI;
use plotters::prelude::*;
use ndarray::prelude::*;
use ndarray_linalg::*;
use qm::*;

extern crate csv;
extern crate ndarray_csv;
use csv::{ReaderBuilder};
use ndarray_csv::{Array2Reader};
use std::error::Error;
use std::fs::File;

pub type VecC64 = ndarray::Array1<c64>;
pub type MatrixC64 = ndarray::Array2<c64>;

In [19]:
pub fn read_c64_array(csv_file: String, array_size: usize) -> Result<MatrixC64, Box<dyn Error>> {

    let file = File::open(csv_file)?;
    let mut reader = ReaderBuilder::new().has_headers(false).from_reader(file);
    let array_read: MatrixC64 = reader.deserialize_array2((array_size, array_size))?;
    Ok(array_read)
}

pub fn read_f64_vector(csv_file: String) -> Result<VecF64, Box<dyn Error>> {

    let file = File::open(csv_file)?;
    let mut reader = ReaderBuilder::new()
        .from_reader(file);

    let v : Vec<f64> = reader.headers()?
        .into_iter()
        .map(|s| s.parse::<f64>().unwrap()) // to f64
        .collect(); // create a vec      
    
    let arr = Array::from(v);
    Ok(arr)
}

let signal: VecF64 = array![1.4446824684143E-06,1.49734123420715E-06,1.54999999999999E-06,1.60265876579284E-06,1.65531753158569E-06];
let idler: VecF64 = array![1.45728222027807E-06,1.51093173528974E-06,1.56458125030141E-06,1.61823076531308E-06,1.67188028032476E-06];
let jsa: MatrixC64 = read_c64_array("jsa_data5.csv".to_string(),5).unwrap();
let dt = 1.0e+06_f64;
let norm = find_two_source_hom(signal.clone(), idler.clone(), jsa.clone(), dt);
println!("norm values are {:?}", norm);
let times = Array::linspace(-300.0e-15_f64, 300.0e-15_f64, 100);
let mut points_time: Vec<f64> = Vec::new();
let mut points_ss: Vec<f64> = Vec::new();
let mut points_ii: Vec<f64> = Vec::new();

for i in 0..99{
    let rate = find_two_source_hom(signal.clone(), idler.clone(), jsa.clone(), times[i]);
    let rate_ss = (0.5*rate.0)/norm.0;
    let rate_ii = (0.5*rate.1)/norm.1;
    points_time.push(times[i]*1.0e+15_f64);
    points_ss.push(rate_ss);
    points_ii.push(rate_ii);
}
println!("ss is {:?}", points_ss);
println!("ii is {:?}", points_ii);
let figure = evcxr_figure((600, 400), |root| {
    root.fill(&WHITE);
    let mut chart = ChartBuilder::on(&root)
        .caption("Huong-Ou-Mandel graph for four photons", ("Arial", 20).into_font())
        .margin(5)
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(-350_f64..350_f64, 0_f64..1.1_f64)?;
    
    chart
        .configure_mesh()
        .x_desc("Time Delay (fs)")
        .y_desc("Coincidence Probability")
        .draw()?;
    
    chart
        .draw_series(LineSeries::new(
            points_time.clone().into_iter().zip(points_ss.into_iter()), &RED,))?
        .label("signal-signal")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &RED));        
    
    chart.draw_series(LineSeries::new(
        points_time.clone().into_iter().zip(points_ii.into_iter()), &BLUE,))?
        .label("idler-idler")
        .legend(|(x, y)| PathElement::new(vec![(x, y), (x + 20, y)], &BLUE));    

    chart
    .configure_series_labels()
    .background_style(&WHITE.mix(0.8))
    .border_style(&BLACK)
    .draw()?;
    Ok(())
});
figure

norm values are (25257522287895550000000000000000000000000000000000000000000000.0, 25486631298438882000000000000000000000000000000000000000000000.0, 25432350569135040000000000000000000000000000000000000000000000.0)
ss is [1.069441904975776, 1.0763009987011891, 1.0810781916962218, 1.0837509583406675, 1.0843066985292422, 1.0827427966811853, 1.0790666336115668, 1.0732955512513118, 1.0654567704261715, 1.055587262122992, 1.043733572883756, 1.0299516051738804, 1.0143063537709842, 0.9968715994141677, 0.977729561142102, 0.9569705089312919, 0.9346923384240835, 0.9110001097092716, 0.8860055522863551, 0.8598265385068188, 0.8325865279413506, 0.8044139852691763, 0.7754417744230775, 0.7458065318492744, 0.7156480218533056, 0.6851084770994775, 0.6543319274107782, 0.623463520077062, 0.5926488349209739, 0.5620331973930927, 0.5317609929703663, 0.5019749861156043, 0.47281564702174095, 0.44442048931412, 0.41692342181873693, 0.39045411742588865, 0.3651374019885502, 0.3410926660943919, 0.3184333024407298, 0.

Huong-Ou-Mandel graph for four photons
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Coincidence Probability
 
 
Time Delay (fs)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.2
 
 
 
0.4
 
 
 
0.6
 
 
 
0.8
 
 
 
1.0
 
 
 
 
-300.0
 
 
 
-200.0
 
 
 
-100.0
 
 
 
0.0
 
 
 
100.0
 
 
 
200.0
 
 
 
300.0
 
 
<polyline fill="none" opacity="1" stroke="#FF0000" stroke-width="1" points="93,38 98,36 102,35 107,34 112,34 116,34 121,35 126,37 130,39 135,42 140,46 145,49 149,54 154,59 159,64 163,70 168,77 173,83 177,91 182,98 187,106 191,114 196,122 201,131 205,139 210,148 215,157 219,166 224,174 229,183 233,192 238,200 243,209 247,217 252,224 257,232 261,239 266,246 271,253 275,259 280,264 285,269 289,274 294,278 299,282 303,285 308,287 313,289 317,290 322,291 327,291 332,290 336,289 341,287 346,285 350,282 355,278 360,274 364,269 369,264 374,259 378,253 383,246 388,239 392,232 397,224 402,217 406,209 411,200 416,192 420,183 425,174 430,166 434,157 439,148 444,139 448,131 453,122 458,114 462,106 467,98 472,91 476,83 481,77 486,70 490,64 495,59 500,54 505,49 509,46 514,42 519,39 523,37 528,35 533,34 537,34 542,34 547,35 551,36 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="93,41 98,39 102,38 107,37 112,37 116,37 121,38 126,40 130,42 135,45 140,48 145,52 149,57 154,62 159,67 163,73 168,79 173,86 177,93 182,100 187,108 191,116 196,124 201,133 205,141 210,150 215,158 219,167 224,176 229,185 233,193 238,201 243,210 247,218 252,226 257,233 261,240 266,247 271,253 275,259 280,265 285,270 289,275 294,279 299,282 303,285 308,287 313,289 317,290 322,291 327,291 332,290 336,289 341,287 346,285 350,282 355,279 360,275 364,270 369,265 374,259 378,253 383,247 388,240 392,233 397,226 402,218 406,210 411,201 416,193 420,185 425,176 430,167 434,158 439,150 444,141 448,133 453,124 458,116 462,108 467,100 472,93 476,86 481,79 486,73 490,67 495,62 500,57 505,52 509,48 514,45 519,42 523,40 528,38 533,37 537,37 542,37 547,38 551,39 "/>
 
 
 
signal-signal
 
 
idler-idler